# Modelo de Logit Multinomial (MNL): Experimentos de Elección

En este notebook, estimaremos un modelo Logit Multinomial (MNL) para conocer las preferencias por un modo innovador de transporte en Suiza: el Swissmetro. Swissmetro es un tren de cercanias que pretende servir como alternativa de media distancia a modos de transporte convencionales como automovil y tren.

## Parte 1: Preparacion de los datos

### Importar modulos

El siguiente codigo permite importar los siguientes paquetes de Python:

- `Pandas`: Permite importar y manejar bases de datos
- `Biogeme`: Estimar modelos de eleccion

In [ ]:
# Instalar paquetes recomendados
!pip3 install biogeme

In [10]:
# Importar modulos

import pandas as pd

import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta


### Abrir base de datos

La base de datos `swissmetro.dat` es un archivo de texto que contiene la siguiente informacion relevante:

- `ID`: Identificador de cada individuo que responde la encuesta
- `PURPOSE`: Proposito del viaje (1 = trabajo / 3 = negocios)
- `CHOICE`: Alternativa elegida en el experimento (0 = desconocido / 1 = tren / 2 = Swissmetro / 3 = automovil)

Los atributos de las alternativas son:

- `TRAIN_TT`: Tiempo de viaje del tren
- `TRAIN_CO`: Costo de viaje del tren
- `TRAIN_HE`: Frecuencia del tren (1 tren cada `TRAIN_HE` minutos)
- `SM_TT`: Tiempo de viaje del Swissmetro
- `SM_CO`: Costo de viaje del Swissmetro
- `SM_HE`: Frecuencia del Swissmetro (1 frecuencia cada `SM_HE` minutos)
- `CAR_TT`: Tiempo de viaje del automovil
- `CAR_CO`: Costo de viaje del automovil

Ademas, la base cuenta con las variables `TRAIN_AV`, `SM_AV` y `CAR_AV` que indican si la situacion de eleccion contiene la alternativa del tren, Swissmetro y automovil respectivamente.

El siguiente codigo abre y presenta la base de datos:

In [11]:
df = pd.read_table('https://raw.githubusercontent.com/ighdez/taller_UNALM/main/taller_ee/swissmetro.dat')
df

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,3,1,1,1192,4,1,7,1,0,5,...,148,13,30,93,17,30,0,156,56,2
10724,3,1,1,1192,4,1,7,1,0,5,...,148,12,30,96,16,10,0,96,70,3
10725,3,1,1,1192,4,1,7,1,0,5,...,148,16,60,93,16,20,0,96,56,3
10726,3,1,1,1192,4,1,7,1,0,5,...,178,16,30,96,17,30,0,96,91,2


### Inicializar la base de datos y excluir variables

El siguiente codigo inicializa la base de datos:

In [12]:
database = db.Database('swissmetro', df)
globals().update(database.variables)

Solo utilizaremos la informacion de las personas que viajan con proposito de trabajo o negocios. Ademas, excluiremos las observaciones con elecciones desconocidas:

In [13]:
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
database.remove(exclude)

### Definir parametros:

Definiremos tres parametros de atributos:

- `B_COST`: Parametro asociado al costo de viaje
- `B_TIME`: Parametro asociado al tiempo de viaje
- `B_HE`: Parametro asociado a la frecuencia del transporte publico

Ademas, definiremos tres constantes por alternativa:

- `ASC_TRAIN`: Constante para la alternativa tren.
- `ASC_SM`: Constante para la alternativa Swissmetro. Sera la categoria base, por lo que sera fija en cero (no estimada)
- `ASC_CAR`: Constante para alternativa automovil.

In [14]:
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 1)
B_TIME = Beta('B_TIME', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)
B_HE = Beta('B_HE', 0, None, None, 0)

### Escalar variables

Para facilidad de interpretacion y estimacion, se recomenda escalar variables con rango amplio. El siguiente codigo permite definir nuevas variables escaladas:

In [15]:
TRAIN_TT_SCALED = TRAIN_TT / 100
TRAIN_COST_SCALED = TRAIN_CO / 100
SM_TT_SCALED = SM_TT / 100
SM_COST_SCALED = SM_CO / 100
CAR_TT_SCALED = CAR_TT / 100
CAR_CO_SCALED = CAR_CO / 100

### Crear funciones de utilidad

En este caso, crearemos funciones de utilidad para cada alternativa. Cada funcion de utilidad esta definida por el tiempo y costo de viaje, mas una constante. Ademas, las funciones de utilidad para tren y Swissmetro tienen frecuencia como atributo adicional.

Recordar que cada funcion de utilidad debe ir asociada a una alternativa de la variable de eleccion (`CHOICE`):

In [16]:
# Crear funciones de utilidad
V1 = ASC_TRAIN + B_TIME * TRAIN_TT_SCALED + B_COST * TRAIN_COST_SCALED + B_HE * TRAIN_HE
V2 = ASC_SM + B_TIME * SM_TT_SCALED + B_COST * SM_COST_SCALED + B_HE * SM_HE
V3 = ASC_CAR + B_TIME * CAR_TT_SCALED + B_COST * CAR_CO_SCALED

# Asociar funciones de utilidad a alternativas
V = {1: V1, 2: V2, 3: V3}



### Crear condiciones de disponibilidad.

En esta base de datos, algunos encuestados enfrentaron menos de tres alternativas. Las variables  `TRAIN_AV`, `SM_AV` y `CAR_AV` indican si la situacion de eleccion contiene la alternativa del tren, Swissmetro y automovil respectivamente. El siguiente codigo permite asociar alternativas con las variables de disponibilidad:

In [17]:
av = {1: TRAIN_AV, 2: SM_AV, 3: CAR_AV}

### Inicializar y estimar el modelo Logit Multinomial

El modelo Logit Multinomial requiere de las funciones de utilidad, las condiciones de disponibilidad y la variable de eleccion. El siguiente codigo permite inicializar y estimar el modelo:

In [18]:
logprob = models.loglogit(V, av, CHOICE)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'mnl'

results = biogeme.estimate()

### Presentacion de resultados

El siguiente codigo permite estimar resultados:

In [19]:
pandasResults = results.getEstimatedParameters()
pandasResults

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,-0.032371,0.046376,-0.698002,4.851757e-01,0.062790,-0.515544,6.061729e-01
ASC_TRAIN,-0.365970,0.069251,-5.284703,1.259087e-07,0.090590,-4.039856,5.348410e-05
B_COST,0.006203,0.002621,2.366395,1.796225e-02,0.002210,2.807544,4.992087e-03
B_HE,-0.005225,0.000960,-5.442956,5.240349e-08,0.000976,-5.354213,8.592946e-08
B_TIME,-1.153740,0.054267,-21.260313,0.000000e+00,0.096007,-12.017226,0.000000e+00


## Ejercicios:

1. Calcule las disposiciones a pagar por menor tiempo de viaje y por cambios en la frecuencia de los medios de transporte publicos.
2. Re-estime el modelo excluyendo la variable de frecuencia de transporte publico. Analice los cambios en la disposicion a pagar por menor tiempo de viaje en ambos modelos.